### Landing To Bronze


In [1]:
today_date = '9999-09-09'
source_account = 'adlsgen7428'
source_container = 'fbproject'
src_relative_path = 'landing'

StatementMeta(, 06e7d64a-49f5-4ac7-8332-d54f23bfb93d, 3, Finished, Available, Finished)

In [2]:


adls_path = 'abfss://%s@%s.dfs.core.windows.net/%s' % (source_container,source_account,src_relative_path)
partition_path = f"/Processing_Date={today_date}/"
compelte_path = adls_path + partition_path
print('source storage path is',compelte_path)

StatementMeta(, 06e7d64a-49f5-4ac7-8332-d54f23bfb93d, 4, Finished, Available, Finished)

source storage path is abfss://fbproject@adlsgen7428.dfs.core.windows.net/landing/Processing_date=2025-07-05/


In [3]:

from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, ArrayType

# Define the schema
schema = StructType([
    StructField("Student_ID", StringType(), True),
    StructField("Name", StringType(), True),
    StructField("Age", IntegerType(), True),
    StructField("Gender", StringType(), True),
    StructField("Grade_Level", StringType(), True),
    StructField("Course_ID", StringType(), True),
    StructField("Course_Name", StringType(), True),
    StructField("Enrollment_Date", StringType(), True),
    StructField("Completion_Date", StringType(), True),
    StructField("Status", StringType(), True),
    StructField("Final_Grade", StringType(), True),
    StructField("Attendance_Rate", DoubleType(), True),
    StructField("Time_Spent_on_Course_hrs", DoubleType(), True),
    StructField("Assignments_Completed", IntegerType(), True),
    StructField("Quizzes_Completed", IntegerType(), True),
    StructField("Forum_Posts", IntegerType(), True),
    StructField("Messages_Sent", IntegerType(), True),
    StructField("Quiz_Average_Score", DoubleType(), True),
    StructField("Assignment_Scores", StringType(), True),
    StructField("Assignment_Average_Score", DoubleType(), True),
    StructField("Project_Score", DoubleType(), True),
    StructField("Extra_Credit", DoubleType(), True),
    StructField("Overall_Performance", DoubleType(), True),
    StructField("Feedback_Score", DoubleType(), True),
    StructField("Parent_Involvement", StringType(), True),
    StructField("Demographic_Group", StringType(), True),
    StructField("Internet_Access", StringType(), True),
    StructField("Learning_Disabilities", StringType(), True),
    StructField("Preferred_Learning_Style", StringType(), True),
    StructField("Language_Proficiency", StringType(), True),
    StructField("Participation_Rate", StringType(), True),
    StructField("Completion_Time_Days", IntegerType(), True),
    StructField("Performance_Score", DoubleType(), True),
    StructField("Course_Completion_Rate", DoubleType(), True)
])

StatementMeta(, 06e7d64a-49f5-4ac7-8332-d54f23bfb93d, 5, Finished, Available, Finished)

In [4]:
df = spark.read.format('csv').schema(schema).option('header','true').load(compelte_path)

print("Reading data of:", partition_path)

df.createOrReplaceTempView('new_data')

StatementMeta(, 06e7d64a-49f5-4ac7-8332-d54f23bfb93d, 6, Finished, Available, Finished)

Reading data of: /Processing_date=2025-07-05/


In [5]:
fabric_bronze_path = f"abfss://{workspace}@onelake.dfs.fabric.microsoft.com/LH_Bronze.Lakehouse/Tables/bronze_data"
print(fabric_bronze_path)

StatementMeta(, 06e7d64a-49f5-4ac7-8332-d54f23bfb93d, 7, Finished, Available, Finished)

abfss://New_Project@onelake.dfs.fabric.microsoft.com/LH_Bronze.Lakehouse/Tables/bronze_data


In [6]:
fabric_bronze_path = f"abfss://{workspace}@onelake.dfs.fabric.microsoft.com/LH_Bronze.Lakehouse/Tables/bronze_data"
try:
    spark.read.format('delta').load(fabric_bronze_path).createOrReplaceTempView('bronze_data')
except:
    create_table = f"""CREATE TABLE IF NOT EXISTS bronze_data(
        Student_ID STRING,
        Name STRING,
        Age INT,
        Gender STRING,
        Grade_Level STRING,
        Course_ID STRING,
        Course_Name STRING,
        Enrollment_Date STRING,
        Completion_Date STRING,
        Status STRING,
        Final_Grade STRING,
        Attendance_Rate DOUBLE,
        Time_Spent_on_Course_hrs DOUBLE,
        Assignments_Completed INT,
        Quizzes_Completed INT,
        Forum_Posts INT,
        Messages_Sent INT,
        Quiz_Average_Score DOUBLE,
        Assignment_Scores STRING,
        Assignment_Average_Score DOUBLE,
        Project_Score DOUBLE,
        Extra_Credit DOUBLE,
        Overall_Performance DOUBLE,
        Feedback_Score DOUBLE,
        Parent_Involvement STRING,
        Demographic_Group STRING,
        Internet_Access STRING,
        Learning_Disabilities STRING,
        Preferred_Learning_Style STRING,
        Language_Proficiency STRING,
        Participation_Rate STRING,
        Completion_Time_Days INT,
        Performance_Score DOUBLE,
        Course_Completion_Rate DOUBLE,
        Processing_Date DATE )"""
    spark.sql(create_table)
    spark.read.format('delta').load(fabric_bronze_path).createOrReplaceTempView('bronze_data')


StatementMeta(, 06e7d64a-49f5-4ac7-8332-d54f23bfb93d, 8, Finished, Available, Finished)

In [11]:
sql_statement = f"""merge into bronze_data as target
                    USING new_data as source
                    on target.Student_ID = source.Student_ID and target.Course_ID = source.Course_Id

                    when matched then
                    update set
                            target.Name = source.Name,
                            target.Age = source.Age,
                            target.Gender = source.Gender,
                            target.Grade_Level = source.Grade_Level,
                            target.Course_Name = source.Course_Name,
                            target.Enrollment_Date = source.Enrollment_Date,
                            target.Completion_Date = source.Completion_Date,
                            target.Status = source.Status,
                            target.Final_Grade = source.Final_Grade,
                            target.Attendance_Rate = source.Attendance_Rate,
                            target.Time_Spent_on_Course_hrs = source.Time_Spent_on_Course_hrs,
                            target.Assignments_Completed = source.Assignments_Completed,
                            target.Quizzes_Completed = source.Quizzes_Completed,
                            target.Forum_Posts = source.Forum_Posts,
                            target.Messages_Sent = source.Messages_Sent,
                            target.Quiz_Average_Score = source.Quiz_Average_Score,
                            target.Assignment_Scores = source.Assignment_Scores,
                            target.Assignment_Average_Score = source.Assignment_Average_Score,
                            target.Project_Score = source.Project_Score,
                            target.Extra_Credit = source.Extra_Credit,
                            target.Overall_Performance = source.Overall_Performance,
                            target.Feedback_Score = source.Feedback_Score,
                            target.Parent_Involvement = source.Parent_Involvement,
                            target.Demographic_Group = source.Demographic_Group,
                            target.Internet_Access = source.Internet_Access,
                            target.Learning_Disabilities = source.Learning_Disabilities,
                            target.Preferred_Learning_Style = source.Preferred_Learning_Style,
                            target.Language_Proficiency = source.Language_Proficiency,
                            target.Participation_Rate = source.Participation_Rate,
                            target.Completion_Time_Days = source.Completion_Time_Days,
                            target.Performance_Score = source.Performance_Score,
                            target.Course_Completion_Rate = source.Course_Completion_Rate,
                            target.Processing_Date = '{today_date}'

                    when not matched then

                    insert (Student_ID, Name, Age, Gender, Grade_Level, Course_ID, Course_Name, Enrollment_Date, Completion_Date,
                                Status, Final_Grade, Attendance_Rate, Time_Spent_on_Course_hrs, Assignments_Completed, Quizzes_Completed,
                                Forum_Posts, Messages_Sent, Quiz_Average_Score, Assignment_Scores, Assignment_Average_Score, Project_Score,
                                Extra_Credit, Overall_Performance, Feedback_Score, Parent_Involvement, Demographic_Group, Internet_Access,
                                Learning_Disabilities, Preferred_Learning_Style, Language_Proficiency, Participation_Rate, Completion_Time_Days,
                                Performance_Score, Course_Completion_Rate, Processing_Date) 
                                values
                                (source.Student_ID, source.Name, source.Age, source.Gender, source.Grade_Level, source.Course_ID, source.Course_Name, 
                                source.Enrollment_Date, source.Completion_Date, source.Status, source.Final_Grade, source.Attendance_Rate, 
                                source.Time_Spent_on_Course_hrs, source.Assignments_Completed, source.Quizzes_Completed, source.Forum_Posts, 
                                source.Messages_Sent, source.Quiz_Average_Score, source.Assignment_Scores, source.Assignment_Average_Score, 
                                source.Project_Score, source.Extra_Credit, source.Overall_Performance, source.Feedback_Score, source.Parent_Involvement, 
                                source.Demographic_Group, source.Internet_Access, source.Learning_Disabilities, source.Preferred_Learning_Style, 
                                source.Language_Proficiency, source.Participation_Rate, source.Completion_Time_Days, source.Performance_Score, 
                                source.Course_Completion_Rate, '{today_date}') """


StatementMeta(, 06e7d64a-49f5-4ac7-8332-d54f23bfb93d, 13, Finished, Available, Finished)